<a href="https://colab.research.google.com/github/MoRamadan253/Reddit_spam_detection/blob/main/Reddit_spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xgboost

In [ ]:
import re
import pickle
import sklearn
import pandas as pd
import numpy as np
import nltk 
from bokeh.io import output_notebook
output_notebook()

from pathlib import Path

# some seeting for pandas and hvplot

pd.options.display.max_columns = 100
pd.options.display.max_rows = 300
pd.options.display.max_colwidth = 100
np.set_printoptions(threshold=2000)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import PredefinedSplit

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data = pd.read_csv("xy_train.csv")
dftest = pd.read_csv('x_test.csv')


Since its classification either fake data or not, means its 1 or 0, so the 2 here is an outlier and can be dropped.

In [ ]:
data['label'].value_counts(dropna=False)

0    32172
1    27596
2      232
Name: label, dtype: int64

In [ ]:
data = data[data.label != 2]

In [ ]:
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words("english"))

def clean_text_stem(text):

    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    words_filtered = [
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def clean_text_lemm(text):

    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    words_filtered = [
            lemmatizer.lemmatize(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

In [ ]:
def clean_char(text):

    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]


    text_clean = " ".join(words_tokens_lower)
    return text_clean

In [ ]:
data["text_clean"] = data["text"].map(lambda x: clean_text_stem(x))
dftest["text_clean"] = dftest["text"].map(lambda x: clean_text_stem(x))

In [ ]:
train, test = train_test_split(data, random_state=1, test_size=0.25, shuffle=True)

X_train = train["text_clean"]
Y_train = train["label"]
X_test = test["text_clean"]
Y_test = test["label"]
print(X_train.shape)
print(X_test.shape)

(44826,)
(14942,)


In [ ]:
X_train2, X_val, y_train2, y_val = train_test_split(
    X_train, Y_train, train_size = 0.8, stratify = Y_train, random_state = 2022)
split_index = [-1 if x in X_train2.index else 0 for x in X_train.index]
pds = PredefinedSplit(test_fold = split_index)

In [ ]:
full_pipline = Pipeline(
    steps=[
        ("tfidf", TfidfVectorizer(analyzer="word",norm="l2")),
        ('my_classifier', XGBClassifier(objective='binary:logistic', seed=1),
        )
    ]
)

In [ ]:
params = {
    "tfidf__ngram_range": [(1,2),(1,3),(1,4)],
    "tfidf__max_df": [0.3,0.4,0.5,0.6,0.8],
    "tfidf__min_df": [13,14,15,16,17],
}

pipe_clf = GridSearchCV(
    full_pipline, params, n_jobs=-1,cv=pds, scoring="roc_auc")
pipe_clf.fit(X_train, Y_train)
print('best score {}'.format(pipe_clf.best_score_))
print('best params {}'.format(pipe_clf.best_params_))

C:\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:24:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
best score 0.8301098383723431
best params {'tfidf__max_df': 0.3, 'tfidf__min_df': 13, 'tfidf__ngram_range': (1, 2)}


In [ ]:
param_grid2 = {
    "tfidf__ngram_range": [(1, 2)],
    "tfidf__max_df": [0.3],
    "tfidf__min_df": [13],
    'my_classifier__learning_rate': [0.3,0.35,0.4],
    'my_classifier__max_depth': [45,50],
    'my_classifier__min_child_weight':[5,7],
    'my_classifier__gamma': [1,2],
    'my_classifier__colsample_bytree': [1],
}


In [ ]:
grid_searchxgb =  GridSearchCV(
    full_pipline, param_grid2, cv=pds, verbose=1, n_jobs=2)
grid_searchxgb.fit(X_train, Y_train)
print('best score {}'.format(grid_searchxgb.best_score_))
print('best params {}'.format(grid_searchxgb.best_params_))

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[13:28:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
best score 0.7657818425161722
best params {'my_classifier__colsample_bytree': 1, 'my_classifier__gamma': 2, 'my_classifier__learning_rate': 0.3, 'my_classifier__max_depth': 50, 'my_classifier__min_child_weight': 5, 'tfidf__max_df': 0.3, 'tfidf__min_df': 13, 'tfidf__ngram_range': (1, 2)}


In [ ]:
rand_searchxgb =  RandomizedSearchCV(
    full_pipline, param_grid2, cv=pds, verbose=1, n_jobs=2)
rand_searchxgb.fit(X_train, Y_train)
print('best score {}'.format(rand_searchxgb.best_score_))
print('best params {}'.format(rand_searchxgb.best_params_))

Fitting 1 folds for each of 10 candidates, totalling 10 fits
[14:05:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
best score 0.7629935311175552
best params {'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 13, 'tfidf__max_df': 0.3, 'my_classifier__min_child_weight': 5, 'my_classifier__max_depth': 50, 'my_classifier__learning_rate': 0.35, 'my_classifier__gamma': 2, 'my_classifier__colsample_bytree': 1}
